In [1]:
!pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install pyngrok

In [3]:
pip install transformers

In [4]:
%%writefile main.py
from typing import Optional
from pydantic import BaseModel
from fastapi import FastAPI
from datetime import datetime
import numpy as np
from transformers import pipeline

app = FastAPI()


@app.get('/calcular_imc')
def calcular_imc(peso_kg: float, altura_m: float):
    imc = peso_kg / (altura_m ** 2)
    return {'IMC': imc}


@app.get("/calcular_edad/")
def calcular_edad(fecha_nacimiento: str):
    try:
        fecha_nacimiento = datetime.strptime(fecha_nacimiento, "%Y-%m-%d")
        hoy = datetime.now()
        edad = hoy.year - fecha_nacimiento.year
        return {"Su edad es": edad}
    except ValueError:
        return {"error": "Formato de fecha incorrecto. Debe ser 'YYYY-MM-DD'."}

@app.post("/calcular_inversa/")
def calcular_inversa(matriz: list[list[float]]) -> dict:
    try:
        matriz_np = np.array(matriz)
        inversa = np.linalg.inv(matriz_np)
        return {"La matriz inversa es": inversa.tolist()}
    except np.linalg.LinAlgError:
        return {"error": "La matriz no es invertible."}


# Inicializa el modelo de pregunta-respuesta
qa_model = pipeline("question-answering", model="timpal0l/mdeberta-v3-base-squad2")

@app.post("/qa")
def question_answer(question: str, context: str):
    try:
        # Realiza la predicción
        result = qa_model(question=question, context=context)
        return {"answer": result["answer"]}
    except Exception as e:
        return {"error": str(e)}


# Inicializa el modelo de traducción de inglés a alemán
translation_model = pipeline("translation",
                              model="google-t5/t5-base"

                            )

@app.post("/translate")
def translate_text(text: str):
    try:
        # Realiza la traducción
        translated_text = translation_model(text)
        return {"translated_text": translated_text[0]["translation_text"]}
    except Exception as e:
        return {"error": str(e)}

Writing main.py


In [5]:
import nest_asyncio
from pyngrok import ngrok, conf
NGROK_TOKEN = "2gQ4sJufZCkp3C97yTaF7jcyyQb_4QDEjvNJupZzvyLBbYcGW"
conf.get_default().auth_token = NGROK_TOKEN

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

Public URL: https://8c81-35-232-139-230.ngrok-free.app


In [6]:
! uvicorn main:app --port 8000

2024-05-19 10:35:09.963401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 10:35:09.963486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 10:35:09.965091: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-19 10:35:11.458714: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
config.json: 100% 879/879 [00:00<00:00, 3.51MB/s]
model.safetensors: 100% 1.11G/1.11G [00:12<00:00, 89.6MB/s]
tokenizer_config.json: 100% 453/453 [00:00<00:00, 2.21MB/s]
tokenizer.json: 100% 16.3M/16.3M [00:00<00:00, 92.6MB/s]
added_tokens.j